In [17]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [18]:
import pandas as pd

In [20]:
df = (pd.read_csv(r"/content/drive/MyDrive/Depression/Dataset/Updated_Dataset/twitter_data.csv"))
# df = (pd.read_csv("../../../Dataset/Updated_Dataset/twitter_data.csv"))
df.head()

,tweet,label
0,the real reason why you be sad you be attach t...,1
1,my biggest problem be overthinking everything,1
2,the worst sadness be the sadness you have teac...,1
3,i cannot make you understand i cannot make any...,1
4,i do not think anyone really understand how ti...,1


In [43]:
from transformers import pipeline
from tqdm import tqdm

emotion_classifier = pipeline("text-classification", model="j-hartmann/emotion-english-distilroberta-base", device=0)

def extract_emotions_batch(df, tweet_column, batch_size=32):
    """
    Extracts emotions from tweets using the emotion classifier in batch mode.

    Args:
    df (pd.DataFrame): DataFrame containing the tweets.
    tweet_column (str): Name of the column containing tweets.
    batch_size (int): Number of tweets to process in each batch.

    Returns:
    pd.DataFrame: Original DataFrame with additional columns for emotions and confidence scores.
    """
    df = df.copy()
    df = df[df[tweet_column].notnull()]
    df[tweet_column] = df[tweet_column].astype(str)

    tweets = df[tweet_column].tolist()
    emotions = []

    for i in tqdm(range(0, len(tweets), batch_size)):
        batch = tweets[i:i + batch_size]
        results = emotion_classifier(batch, truncation=True, top_k=1)
        for result in results:
            emotions.append(result[0]['label'])

    df['aspect'] = emotions

    return df

In [44]:
df_with_emotions = extract_emotions_batch(df, 'tweet')

100%|██████████| 243/243 [00:44<00:00,  5.46it/s]


In [45]:
df_with_emotions.head()

,tweet,label,aspect
0,the real reason why you be sad you be attach t...,1,sadness
1,my biggest problem be overthinking everything,1,anger
2,the worst sadness be the sadness you have teac...,1,sadness
3,i cannot make you understand i cannot make any...,1,sadness
4,i do not think anyone really understand how ti...,1,joy


In [46]:
df_with_emotions.to_csv(r"/content/drive/MyDrive/Depression/Dataset/Updated_Dataset/twitter_data_with_emotions.csv", index=False)